## PATH SETUP

In [1]:
# MUST be first cell: set multiprocessing method for Windows
import torch.multiprocessing as mp
try:
    mp.set_start_method("spawn", force=True)
except RuntimeError:
    pass  # Already set

In [2]:
import os
import sys
from pathlib import Path

# FOR LOCAL USE THIS LINES
current = Path.cwd()
src_path = current / "src" if (current / "src").exists() else current.parent

# FOR COLAB USE THIS LINE INSTEAD
# BRANCH_NAME = "main"  # Change this to switch branches
# !git clone -b {BRANCH_NAME} https://github.com/MatteoCamillo-code/GeoLoc-CVCS.git
# !cd /content/GeoLoc-CVCS && git pull origin {BRANCH_NAME} && cd ..
# src_path = Path("/content/GeoLoc-CVCS/src").resolve()

sys.path.insert(0, str(src_path))

from utils.paths import find_project_root

# Set working directory and sys.path properly
project_root = find_project_root(src_path)
data_dir = project_root / "data"
history_dir = project_root / "outputs" / "history"
os.chdir(project_root)
sys.path.insert(0, str(project_root / "src"))
print("CWD:", Path.cwd())

CWD: F:\InfTech\Prodotti\Python\GeoLocGit\GeoLoc-CVCS


## IMPORT

In [3]:
import pandas as pd
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR

from configs.baseline_multi_head import TrainConfig

cfg = TrainConfig()
if cfg.backbone == "resnet50":
    from torchvision.models import resnet50, ResNet50_Weights
elif cfg.backbone == "inceptionv4":
    import timm

from utils.seed import seed_everything
from utils.metrics import overall_val_acc_from_history
from utils.io import save_json, delete_file
from utils.paths import get_next_version, get_current_version
from training.runner import fit
from training.losses import CrossEntropyWithLabelSmoothing

from src.utils.logging import get_logger
from src.utils.paths import abs_path

from models.multi_head_classifier import MultiHeadClassifier


In [4]:
seed_everything(cfg.seed)

device = cfg.device if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cuda


In [5]:
import kagglehub

path = kagglehub.dataset_download("josht000/osv-mini-129k")
path = path + "/osv5m"
print("Path to dataset files:", path)

image_root = path + "/train_images"


c:\Users\camil\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\camil\.cache\kagglehub\datasets\josht000\osv-mini-129k\versions\1/osv5m


In [6]:
if len(cfg.scenes) == 1 and cfg.scenes[0] == "total":
    train_val_path = data_dir / "metadata/s2-geo-cells/train_val_split_geocells_total.csv"
else:
    train_val_path = data_dir / "metadata/s2-geo-cells/train_val_split_geocells.csv"
cell_centers_dfs = {}
cells_hierarchy_dfs = {}

for scene in cfg.scenes:
    if cfg.same_partitions:
        cell_centers_path = data_dir / f"metadata/s2-geo-cells/cell_center_dataset_total.csv"
        cells_hierarchy_path = data_dir / f"metadata/s2-geo-cells/cell_hierarchy_dataset_total.csv"
    else:
        cell_centers_path = data_dir / f"metadata/s2-geo-cells/cell_center_dataset_{scene}.csv"
        cells_hierarchy_path = data_dir / f"metadata/s2-geo-cells/cell_hierarchy_dataset_{scene}.csv"
    
    cell_centers_df = pd.read_csv(cell_centers_path)
    cells_hierarchy_df = pd.read_csv(cells_hierarchy_path)
    
    # remove duplicates with same cell_id
    cell_centers_df = cell_centers_df.drop_duplicates(subset='cell_id_token')

    # Set S2 cell ID as index (assumes first column or 'cell_id' column)
    if 'cell_id_token' in cell_centers_df.columns:
        cell_centers_df = cell_centers_df.set_index('cell_id_token')
    else:
        # Set first column as index if it contains cell IDs
        cell_centers_df = cell_centers_df.set_index(cell_centers_df.columns[0])
        
    cell_centers_dfs[scene] = cell_centers_df
    cells_hierarchy_dfs[scene] = cells_hierarchy_df

## DATALOADER

In [7]:
from dataset.dataloader_utils import create_dataloaders

IMG_SIZE = 224

# Create all dataloaders with a single function call
loader_dict = create_dataloaders(
    image_root=image_root,
    csv_path=train_val_path,
    batch_size=cfg.batch_size,
    num_workers=cfg.num_workers,
    img_size=IMG_SIZE,
    seed=cfg.seed,
    train_subset_pct=cfg.train_size_pct,
    val_subset_pct=cfg.val_size_pct,
    scenes=cfg.scenes,
    augment=True,
    prefetch_factor=cfg.prefetch_factor,
    persistent_workers=True if cfg.num_workers > 0 else False,
    coarse_label_idx=cfg.coarse_label_idx,
)

## MODEL

In [8]:
models = {}

for sc in cfg.scenes:
    backbone = None
    FEAT_DIM = 0
    if cfg.backbone == "inceptionv4":
        backbone = timm.create_model('inception_v4', pretrained=True)
        FEAT_DIM = 1536  # inception_v4 feature dimension
    elif cfg.backbone == "resnet50":
        weights = ResNet50_Weights.IMAGENET1K_V2
        backbone = resnet50(weights=weights)
        FEAT_DIM = 2048  # resnet50 feature dimension  

    # number of classes depends on partition
    num_classes = list(map(
        lambda idx: len(loader_dict[sc]["label_maps"][f"label_config_{idx + 1}"]),
        cfg.coarse_label_idx
    ))

    backbone = nn.Sequential(
        *list(backbone.children())[:-1],
        nn.Flatten(1)
    )


    backbone = backbone.to(device)
    # Optional: comment out if it causes issues on Windows/your PyTorch version
    # model = torch.compile(model, backend="aot_eager")

    model = MultiHeadClassifier(
        backbone=backbone,
        feat_dim=FEAT_DIM,
        head_dims=num_classes,
        dropout=cfg.dropout,
        coarse_level_idx=cfg.coarse_label_idx,
    ).to(device)
    
    models[sc] = model

    print(f"Output classes {sc}:", num_classes)


Output classes total: [4741, 2508, 1336]


In [9]:
criterion = CrossEntropyWithLabelSmoothing(ignore_index=-1, smoothing=cfg.label_smoothing)

# Create optimizer, scheduler, and scaler for each model
optimizers = {}
schedulers = {}
scalers = {}

for scene, model in models.items():
    optimizers[scene] = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    schedulers[scene] = StepLR(optimizers[scene], step_size=cfg.scheduler_step_size, gamma=cfg.scheduler_gamma)
    scalers[scene] = torch.amp.GradScaler(device=cfg.device, enabled=cfg.amp)

torch.backends.cudnn.benchmark = True
print(f"Initialized training components for {len(models)} scenes: {list(models.keys())}")

Initialized training components for 1 scenes: ['total']


## TRAINING LOOP

In [10]:
# Train each scene's model separately
histories = {}
logger = get_logger(log_file=str(abs_path(cfg.output_dir, "logs", "train.log")))
version = 0

base_name = f"{cfg.model_name}"
full_path = history_dir / f"{base_name}_v{version}.json"

# Store checkpoint paths for later consolidation
checkpoint_paths = {}

for scene in models.keys():
    # Get versioned history path
    base_name = f"{cfg.model_name}_{scene}"
    version = get_next_version(history_dir, base_name)
    history_path = history_dir / f"{base_name}_v{version}.json"
    
    history = fit(
        cfg=cfg,
        model=models[scene],
        data_loader=loader_dict[scene],
        cell_centers=cell_centers_dfs[scene],
        cells_hierarchy=cells_hierarchy_dfs[scene],
        optimizer=optimizers[scene],
        criterion=criterion,
        scaler=scalers[scene],
        use_tqdm=cfg.use_tqdm,
        scheduler=schedulers[scene],
        logger=logger,
        scene=scene,
        history_path=history_path,
        version=version,
    )
    
    histories[scene] = history
    checkpoint_paths[scene] = str(abs_path(cfg.output_dir, "checkpoints", f"{cfg.model_name}_{scene}_v{version}.pt"))


[16:45:48] INFO - Starting training MH_res_w_same_partitions for scene total ...
[16:50:27] INFO - Epoch 1/15 | train loss=6.8985 acc=13.26% | val loss=6.3449 acc=22.39% | geo acc={'acc@1km': '6.55%', 'acc@5km': '24.79%', 'acc@25km': '42.66%', 'acc@100km': '53.80%'} | time=278.3s
[16:54:59] INFO - Epoch 2/15 | train loss=5.8495 acc=33.65% | val loss=5.8967 acc=30.46% | geo acc={'acc@1km': '8.82%', 'acc@5km': '32.20%', 'acc@25km': '52.43%', 'acc@100km': '62.46%'} | time=271.6s
[16:58:50] INFO - Epoch 3/15 | train loss=5.3713 acc=44.86% | val loss=5.6634 acc=34.75% | geo acc={'acc@1km': '10.67%', 'acc@5km': '36.98%', 'acc@25km': '57.61%', 'acc@100km': '66.92%'} | time=230.2s
[17:03:14] INFO - Epoch 4/15 | train loss=5.0660 acc=52.88% | val loss=5.5241 acc=37.89% | geo acc={'acc@1km': '11.98%', 'acc@5km': '39.91%', 'acc@25km': '61.35%', 'acc@100km': '70.31%'} | time=264.2s
[17:07:32] INFO - Epoch 5/15 | train loss=4.8468 acc=59.23% | val loss=5.4198 acc=40.19% | geo acc={'acc@1km': '12.71

In [11]:
# Save consolidated checkpoints from all scenes into a single file
consolidated_checkpoints = {}
for scene, ckpt_path in checkpoint_paths.items():
    ckpt = torch.load(ckpt_path, map_location=cfg.device)
    consolidated_checkpoints[scene] = ckpt

# Save consolidated checkpoint
consolidated_ckpt_path = abs_path(cfg.output_dir, "checkpoints", f"{cfg.model_name}_v{version}.pt")
torch.save(consolidated_checkpoints, consolidated_ckpt_path)
logger.info(f"Saved consolidated checkpoints to {consolidated_ckpt_path}")

# Delete individual scene checkpoints
for ckpt_path in checkpoint_paths.values():
    if Path(ckpt_path).exists():
        Path(ckpt_path).unlink()
        logger.info(f"Deleted {ckpt_path}")

save_json(obj=histories, path=full_path)
for scene in cfg.scenes:
    base_name = f"{cfg.model_name}_{scene}"
    history_path = history_dir / f"{base_name}_v{version}.json"
    delete_file(history_path)

[17:46:55] INFO - Saved consolidated checkpoints to F:\InfTech\Prodotti\Python\GeoLocGit\GeoLoc-CVCS\outputs\checkpoints\MH_res_w_same_partitions_v1.pt
[17:46:55] INFO - Deleted F:\InfTech\Prodotti\Python\GeoLocGit\GeoLoc-CVCS\outputs\checkpoints\MH_res_w_same_partitions_total_v1.pt
